## Auto ML Clasificacion Vinos

In [1]:
import azureml.core
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()

### Preparar Datos

In [2]:
from azureml.core import Dataset

wine_ds = Dataset.get_by_name(ws, name='wine_data')

### Crear Compute Cluster

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "my-ml-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=4)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Crear configuracion AutoML

In [4]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Auto ML exp clasificacion prueba',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = wine_ds,
                             label_column_name='name',
                             experiment_timeout_hours= 1,
                             enable_early_stopping= True,
                             n_cross_validations= 3,
                             num_classes= 3,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=4,
                             featurization='auto'
                             )

### Correr Experimiento

In [5]:
from azureml.core.experiment import Experiment

automl_experiment = Experiment(ws, 'Azure-AutoML-Clasificar-Vinos')
automl_run = automl_experiment.submit(automl_config)
automl_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Azure-AutoML-Clasificar-Vinos,AutoML_4a673ac3-6d78-4a50-a498-3f01e90a25e4,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
Azure-AutoML-Clasificar-Vinos,AutoML_4a673ac3-6d78-4a50-a498-3f01e90a25e4,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

{'runId': 'AutoML_4a673ac3-6d78-4a50-a498-3f01e90a25e4',
 'target': 'my-ml-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-11-06T15:49:53.667732Z',
 'endTimeUtc': '2021-11-06T16:13:36.587951Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'my-ml-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Auto ML exp clasificacion prueba","subscription_id":"030feb6f-715f-420c-90a9-4d556309931c","resource_group":"my_ml_workspace","workspace_name":"my_ml_workspace","region":"eastus","compute_target":"my-ml-clust

### Obtener Mejor Modelo

In [6]:
best_run, fitted_model = automl_run.get_output()
print(best_run)

print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.35.1, current version:1.34.0
Package:azureml-core, training version:1.35.0.post1, current version:1.34.0
Package:azureml-dataprep, training version:2.23.2, current version:2.22.2
Package:azureml-dataprep-rslex, training version:1.21.2, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.35.0, current version:1.34.0
Package:azureml-defaults, training version:1.35.0, current version:1.34.0
Package:azureml-interpret, training version:1.35.0, current version:1.34.0
Package:azureml-mlflow, training version:1.35.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.35.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.35.0, current version:1.34.0
Package:azureml-telemetry, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-runtime, training version:1.35.1, current

Run(Experiment: Azure-AutoML-Clasificar-Vinos,
Id: AutoML_4a673ac3-6d78-4a50-a498-3f01e90a25e4_12,
Type: azureml.scriptrun,
Status: Completed)

Best Run Metrics:
matthews_correlation 0.9681785666361685
precision_score_macro 0.9814814814814815
norm_macro_recall 0.9666666666666665
average_precision_score_micro 0.9960162463718341
balanced_accuracy 0.9777777777777777
f1_score_micro 0.9774011299435029
f1_score_macro 0.9775533108866442
precision_score_weighted 0.9811676082862523
AUC_micro 0.9979412046346835
recall_score_micro 0.9774011299435029
weighted_accuracy 0.9770312948607524
precision_score_micro 0.9774011299435029
AUC_macro 1.0
recall_score_weighted 0.9774011299435029
recall_score_macro 0.9777777777777777
average_precision_score_macro 1.0
accuracy 0.9774011299435029
f1_score_weighted 0.9771728585287907
AUC_weighted 1.0
average_precision_score_weighted 1.0
log_loss 0.45499646475178596
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_4a673ac3-6d78-4a50-a498-3f01e90a25e4_12/accu

### Registrar mejor modelo

In [7]:
best_run.register_model(model_path='outputs/model.pkl', model_name='automl_wine_classification_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})


Model(workspace=Workspace.create(name='my_ml_workspace', subscription_id='030feb6f-715f-420c-90a9-4d556309931c', resource_group='my_ml_workspace'), name=automl_wine_classification_model, id=automl_wine_classification_model:1, version=1, tags={'Training context': 'Auto ML'}, properties={'AUC': '1.0', 'Accuracy': '0.9774011299435029'})

### Eliminar Cluster

In [8]:
training_cluster.delete()